In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# Lab 2: SQL

In this lab, we are going to practice viewing, sorting, grouping, and merging tables with SQL. We will explore the money donated during the 2016 election using the [Federal Election Commission's public records](https://www.fec.gov/data/). You will be connecting to a sqlite database containing the data. The data we will be working with in this lab is quite small (16MB); however, it's taken from a larger database that more is than few GBs.


### Due Date

This assignment is due at **Tuesday, September 8th** at 11:59 pm.

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about this assignment, we ask that you **write your solutions individually**. If you discuss the assignment with others, please **include their names** in the cell below.

**Collaborators:** *list names here*

In [1]:
import pandas as pd
import sqlalchemy
from ds100_utils import fetch_and_cache
from pathlib import Path

# Part I – SQL

### Connecting to SQLite

Download the database to local disk drive; this will take about a minute.

In [2]:
db_url = 'https://s3.amazonaws.com/berkeley-ds100/fec_nyc.sqlite'
db_filename = 'fec_nyc.db'
data_dir = Path('data')
fetch_and_cache(db_url, db_filename, data_dir=data_dir)
db_path = data_dir / db_filename

The following cell will connect to the above database using the SQL Alchemy library. This library is more general than the `sqlite3` module from the standard library because it can connect to several different database management systems, such as MySQL and PostgreSQL. It also supports an advanced feature for generating queries called an [object relational mapper](https://docs.sqlalchemy.org/en/latest/orm/tutorial.html) or ORM, which we won't discuss in this course but is quite useful for application development.

In [3]:
engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()

To check that everything is working we will list the available tables.

In [4]:
engine.table_names()

## Table Descriptions

Here is a list of the tables in the database.  Each table links to the documentation on the [FEC page](https://www.fec.gov/data/browse-data/) for the dataset.

If you'd like, you can consult the FEC page for the descriptions of the tables to find out what the correspondence is.

- [`cand`](https://www.fec.gov/campaign-finance-data/candidate-summary-file-description/): Candidates table. Contains names and party affiliation.
- [`comm`](https://www.fec.gov/campaign-finance-data/committee-summary-file-description/): Committees table. Contains committee names and types.
- [`indiv_sample_nyc`](https://www.fec.gov/campaign-finance-data/contributions-individuals-file-description/): All individual contributions from New York City.

We can see the schemas of these tables with the following code.

In [5]:
for exp in connection.execute('SELECT sql FROM sqlite_master;'):
    print(exp[0])

We can also look at the tables themselves.

The individual donations made by NY residents are shown below.

In [6]:
# We use `LIMIT 5` to avoid loading a huge table.
# Although our tables shouldn't get too large to display,
# this is generally good practice when working in the
# notebook environment.  Jupyter notebooks don't handle
# very large outputs well. 
query = """
SELECT * from indiv_sample_nyc LIMIT 5
"""
pd.read_sql(query, engine)

Similarly, the database of candidates is given below.

In [7]:
query = '''
SELECT *
FROM cand
LIMIT 5
'''
pd.read_sql(query,engine)

And lastly, the table of committees is given below.

In [8]:
query = '''
SELECT *
FROM comm
LIMIT 5
'''
pd.read_sql(query,engine)

In CS61A or CS88, we saw basic queries similar to what we see below, which gives only the id and name of candidates from the Democratic party.

In [9]:
query = '''
SELECT cand_id, cand_name
FROM cand
WHERE cand_pty_affiliation = 'DEM'
LIMIT 5
'''
pd.read_sql(query,engine)

---
## Question 1

For this question we will use SQL to extract data out of the `indiv_sample_nyc` table. The schema for this table is given below.

In [10]:
for exp in connection.execute('SELECT sql FROM sqlite_master WHERE name="indiv_sample_nyc";'):
    print(exp[0])

### Question 1a

Let's start by looking at 2016 election contributions made by Donald Trump, who was a NY resident during that year. Write a SQL statement that will return the cmte_id, transaction_amt, and name for every contribution made by any donor with "DONALD" and "TRUMP" in their name.


Hint: You'll need to use `WHERE name LIKE '%TRUMP%' AND name LIKE '%DONALD%'`.

<!--
BEGIN QUESTION
name: q1a
-->

In [11]:
query_q1a='''
SELECT
    ... 
    ... 
    ...
FROM indiv_sample_nyc
WHERE ...
'''


res_q1a = pd.read_sql(query_q1a, engine)
res_q1a

In [ ]:
grader.check("q1a")

### Question 1b

If we look at the list above, it appears that some donations were not by Donald Trump himself, but instead by an entity called "DONALD J TRUMP FOR PRESIDENT INC". If we look at the list of donors, we see that our query only seems to have picked up one such anomalous name.

In [14]:
res_q1a['name'].value_counts()

Revise your query from above so that the 15 donations made by "DONALD J TRUMP FOR PRESIDENT INC" do not appear. Your resulting table should have 142 rows. 

Hint: Consider using `NOT LIKE`

<!--
BEGIN QUESTION
name: q1b
-->

In [15]:
query_q1b = '''
SELECT
    ... 
    ... 
    ...
FROM indiv_sample_nyc
WHERE ...
'''

res_q1b = pd.read_sql(query_q1b, engine)
res_q1b

In [ ]:
grader.check("q1b")

### Question 1c

As a review of some of the key ideas from CS61A/CS88, let's refine our query above.

We see that many of the contributions above were to the same committee. Create a new SQL query that returns the total amount that Donald Trump contributed to each committee. Your table should have three columns `cmte_id`, `total_amount`, and `num_donations`, where `total_amount` is the total amount contributed to that committee, and `num_donations` is the total number of donations. Your table should be sorted in decreased order of `total_amount`.

Hint: Use `GROUP BY` as covered in CS61A and CS88.

Hint: Use `COUNT(*)` as covered in CS61A and CS88.

Hint: Use `SUM()` as covered in CS61A and CS88.

Hint: Use `ORDER BY` as covered in CS61A and CS88.

Hint: Use `DESC` as covered in CS61A and CS88.

Hint: Your `WHERE` clause should be the same as question 1b.

<!--
BEGIN QUESTION
name: q1c
-->

In [18]:
query_q1c = '''
SELECT
    ... 
    ... 
    ...
FROM indiv_sample_nyc
WHERE ...
...
...
'''


res_q1c = pd.read_sql(query_q1c, engine)
res_q1c

In [ ]:
grader.check("q1c")

### Question 1d

Now we'd like to know the name of each committee, i.e. we want to know the name of the identity that Donald Trump was making contributions to. Modify your query from question 1c so that it also shows the name of each committee. That is, your table should have four columns `cmte_id`, `total_amount`, `num_donations`, and `cmte_nm`.

The names of the committees are given in the `cmte_nm` column of the `comm` table.

Hint: You'll need to join the `indiv_sample_nyc` and `comm` tables.

Hint: One approach is a nested select statement (a.k.a. subquery) as discussed in lecture.

<!--
BEGIN QUESTION
name: q1d
-->

In [22]:
query_q1d = '''
...
'''


res_q1d = pd.read_sql(query_q1d, engine)
res_q1d

In [ ]:
grader.check("q1d")

## Question 2

### Question 2a

Let's select transactions in the `indiv_sample_nyc` table with a `cmte_id` (committee id) that ends in 5. Then, groupby the transactions in the `indiv_sample_nyc` table with the `cmte_id` column, and count how many transactions and how much total amount of contribution there are for each `cmte_id`. Select the top 5 committees ordered by the transaction count.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>committee_id</th>
      <th>total_amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C00575795</td>
      <td>25099091</td>
      <td>110657</td>
    </tr>
    <tr>
      <th>1</th>
      <td>C00000935</td>
      <td>6989835</td>
      <td>36237</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C00035675</td>
      <td>316019</td>
      <td>3417</td>
    </tr>
    <tr>
      <th>3</th>
      <td>C00040535</td>
      <td>492567</td>
      <td>3277</td>
    </tr>
    <tr>
      <th>4</th>
      <td>C00012245</td>
      <td>499659</td>
      <td>2128</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2a
-->

In [27]:
query_q2a='''
SELECT
    ... AS committee_id,
    ... AS total_amount,
    ... AS count
FROM indiv_sample_nyc
...
...
...
...
'''


res_q2a = pd.read_sql(query_q2a, engine)
res_q2a

In [ ]:
grader.check("q2a")

### Question 2b

When you printed the schema for the tables, you may have noticed that both the `cand` and `comm` tables have a `cand_id` column. Let's try joining these two tables on this column.

List the first 5 candidate names (`cand_name`) in reverse lexicographic order by `cand_name`, along with their corresponding committee names. Only select rows that have a matching `cand_id` in both tables.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORN, ROBERT ERWIN</td>
      <td>CONSTITUTIONAL COMMITTEE</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2b
-->

In [31]:
query_q2b='''
SELECT ..., ...
FROM cand c1 ... JOIN comm c2 ON ...
...
...
'''


res_q2b = pd.read_sql(query_q2b, engine)
res_q2b

In [ ]:
grader.check("q2b")

### Question 2c

As in the previous part, list the first 5 candidate names (`cand_name`) by reverse lexicographic order and their corresponding committee names (`cmte_nm`). This time, include all candidate names. If there is no matching `cand_id` in the `comm` table, then `cmte_nm` should be NULL.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cand_name</th>
      <th>cmte_nm</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ZUTLER, DANIEL PAUL MR</td>
      <td>CITIZENS TO ELECT DANIEL P ZUTLER FOR PRESIDENT</td>
    </tr>
    <tr>
      <th>1</th>
      <td>ZUMWALT, JAMES</td>
      <td>ZUMWALT FOR CONGRESS</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ZUKOWSKI, ANDREW GEORGE</td>
      <td>ZUKOWSKI FOR CONGRESS</td>
    </tr>
    <tr>
      <th>3</th>
      <td>ZUCCOLO, JOE</td>
      <td>JOE ZUCCOLO FOR CONGRESS</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ZORNOW, TODD MR</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

<!--
BEGIN QUESTION
name: q2c
-->

In [35]:
query_q2c='''
SELECT ..., ...
FROM cand c1 ... JOIN comm c2 ON ...
...
...
'''


res_q2c = pd.read_sql(query_q2c, engine)
res_q2c

In [ ]:
grader.check("q2c")

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export("lab02.ipynb", pdf=False)